# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 12:29PM,257466,19,ACG-2102492765,ACG North America LLC,Released
1,Feb 15 2023 12:25PM,257465,16,WHC-657560,WHC Labs bvba,Released
2,Feb 15 2023 12:21PM,257464,19,TR-RC-02172023,"GCH Granules USA, Inc.",Released
3,Feb 15 2023 12:20PM,257463,15,PNC415288A,"Person & Covey, Inc.",Released
4,Feb 15 2023 12:20PM,257463,15,PNC415446A,"Person & Covey, Inc.",Released
5,Feb 15 2023 12:20PM,257463,15,PNC428851A,"Person & Covey, Inc.",Released
6,Feb 15 2023 12:20PM,257463,15,PNC415359C,"Person & Covey, Inc.",Released
7,Feb 15 2023 12:20PM,257463,15,PNC415373C,"Person & Covey, Inc.",Released
8,Feb 15 2023 12:20PM,257463,15,PNC425575B,"Person & Covey, Inc.",Released
9,Feb 15 2023 12:20PM,257463,15,PNC425243B,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,257462,Released,12
53,257463,Released,30
54,257464,Released,1
55,257465,Released,1
56,257466,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257462,NaN,NaN,12.0
257463,NaN,NaN,30.0
257464,NaN,NaN,1.0
257465,NaN,NaN,1.0
257466,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257232,0.0,0.0,24.0
257354,1.0,0.0,6.0
257366,0.0,0.0,1.0
257373,0.0,0.0,1.0
257375,1.0,5.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257232,0,0,24
257354,1,0,6
257366,0,0,1
257373,0,0,1
257375,1,5,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257232,0,0,24
1,257354,1,0,6
2,257366,0,0,1
3,257373,0,0,1
4,257375,1,5,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257232,,,24
1,257354,1,,6
2,257366,,,1
3,257373,,,1
4,257375,1,5,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 12:29PM,257466,19,ACG North America LLC
1,Feb 15 2023 12:25PM,257465,16,WHC Labs bvba
2,Feb 15 2023 12:21PM,257464,19,"GCH Granules USA, Inc."
3,Feb 15 2023 12:20PM,257463,15,"Person & Covey, Inc."
33,Feb 15 2023 12:17PM,257462,10,"Digital Brands, LLC"
45,Feb 15 2023 12:06PM,257232,15,"Brookfield Pharmaceuticals, LLC"
69,Feb 15 2023 11:59AM,257461,10,"Senseonics, Incorporated"
70,Feb 15 2023 11:56AM,257460,15,"Brookfield Pharmaceuticals, LLC"
71,Feb 15 2023 11:28AM,257455,10,"Methapharm, Inc."
83,Feb 15 2023 11:27AM,257454,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,,,1
1,Feb 15 2023 12:25PM,257465,16,WHC Labs bvba,,,1
2,Feb 15 2023 12:21PM,257464,19,"GCH Granules USA, Inc.",,,1
3,Feb 15 2023 12:20PM,257463,15,"Person & Covey, Inc.",,,30
4,Feb 15 2023 12:17PM,257462,10,"Digital Brands, LLC",,,12
5,Feb 15 2023 12:06PM,257232,15,"Brookfield Pharmaceuticals, LLC",,,24
6,Feb 15 2023 11:59AM,257461,10,"Senseonics, Incorporated",,,1
7,Feb 15 2023 11:56AM,257460,15,"Brookfield Pharmaceuticals, LLC",,,1
8,Feb 15 2023 11:28AM,257455,10,"Methapharm, Inc.",,,12
9,Feb 15 2023 11:27AM,257454,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,1,,
1,Feb 15 2023 12:25PM,257465,16,WHC Labs bvba,1,,
2,Feb 15 2023 12:21PM,257464,19,"GCH Granules USA, Inc.",1,,
3,Feb 15 2023 12:20PM,257463,15,"Person & Covey, Inc.",30,,
4,Feb 15 2023 12:17PM,257462,10,"Digital Brands, LLC",12,,
5,Feb 15 2023 12:06PM,257232,15,"Brookfield Pharmaceuticals, LLC",24,,
6,Feb 15 2023 11:59AM,257461,10,"Senseonics, Incorporated",1,,
7,Feb 15 2023 11:56AM,257460,15,"Brookfield Pharmaceuticals, LLC",1,,
8,Feb 15 2023 11:28AM,257455,10,"Methapharm, Inc.",12,,
9,Feb 15 2023 11:27AM,257454,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,1,,
1,Feb 15 2023 12:25PM,257465,16,WHC Labs bvba,1,,
2,Feb 15 2023 12:21PM,257464,19,"GCH Granules USA, Inc.",1,,
3,Feb 15 2023 12:20PM,257463,15,"Person & Covey, Inc.",30,,
4,Feb 15 2023 12:17PM,257462,10,"Digital Brands, LLC",12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,1.0,NaN,NaN
1,Feb 15 2023 12:25PM,257465,16,WHC Labs bvba,1.0,NaN,NaN
2,Feb 15 2023 12:21PM,257464,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Feb 15 2023 12:20PM,257463,15,"Person & Covey, Inc.",30.0,NaN,NaN
4,Feb 15 2023 12:17PM,257462,10,"Digital Brands, LLC",12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 12:29PM,257466,19,ACG North America LLC,1.0,0.0,0.0
1,Feb 15 2023 12:25PM,257465,16,WHC Labs bvba,1.0,0.0,0.0
2,Feb 15 2023 12:21PM,257464,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Feb 15 2023 12:20PM,257463,15,"Person & Covey, Inc.",30.0,0.0,0.0
4,Feb 15 2023 12:17PM,257462,10,"Digital Brands, LLC",12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6692984,221.0,14.0,1.0
12,514854,1.0,1.0,0.0
15,2059207,75.0,16.0,22.0
16,257465,1.0,0.0,0.0
19,1544552,41.0,9.0,0.0
21,1029770,4.0,0.0,0.0
22,514789,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6692984,221.0,14.0,1.0
1,12,514854,1.0,1.0,0.0
2,15,2059207,75.0,16.0,22.0
3,16,257465,1.0,0.0,0.0
4,19,1544552,41.0,9.0,0.0
5,21,1029770,4.0,0.0,0.0
6,22,514789,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,221.0,14.0,1.0
1,12,1.0,1.0,0.0
2,15,75.0,16.0,22.0
3,16,1.0,0.0,0.0
4,19,41.0,9.0,0.0
5,21,4.0,0.0,0.0
6,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,221.0
1,12,Released,1.0
2,15,Released,75.0
3,16,Released,1.0
4,19,Released,41.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,1.0,0.0,22.0,0.0,0.0,0.0,0.0
Executing,14.0,1.0,16.0,0.0,9.0,0.0,1.0
Released,221.0,1.0,75.0,1.0,41.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,1.0,0.0,22.0,0.0,0.0,0.0,0.0
1,Executing,14.0,1.0,16.0,0.0,9.0,0.0,1.0
2,Released,221.0,1.0,75.0,1.0,41.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,1.0,0.0,22.0,0.0,0.0,0.0,0.0
1,Executing,14.0,1.0,16.0,0.0,9.0,0.0,1.0
2,Released,221.0,1.0,75.0,1.0,41.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()